<a href="https://colab.research.google.com/github/Thomas-J-Kidd/ECEN4493_transfer_learning_examples/blob/main/transfer_learning_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Necessary Libraries

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt

# Load and Preprocess Your Dataset
For demonstration, let's use the CIFAR-10 dataset, which is directly available in TensorFlow. The dataset contains 60,000 color images in 10 classes, with 6,000 images per class. We will also use Mobilnet V2 as its a lightwieght model

In [17]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from sklearn.model_selection import train_test_split

# Load CIFAR-10 data
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# Preprocess data for MobileNetV2
train_images = preprocess_input(train_images)
test_images = preprocess_input(test_images)


# Load a Pre-Trained Model
Load a pre-trained model without the top layer (since we will be adding our own). We will use ResNet50 as an example:

In [18]:
base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
base_model.trainable = False  # Freeze the base model


# Add Custom Layers
Now, add your custom layers on top of the pre-trained model. For CIFAR-10, you might want a few dense layers with a final layer of 10 units (for 10 classes) with a softmax activation.

In [19]:
model = models.Sequential([
  base_model,
  layers.GlobalAveragePooling2D(),
  layers.Dense(10, activation='softmax'),
])

# Compile the Model
Compile the model with an appropriate optimizer, loss function, and metrics:

In [20]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the Model
Train the model on your dataset. Since we're using a pre-trained model, you might not need as many epochs.

In [21]:
# Use a smaller subset of the data for faster training
_, small_train_images, _, small_train_labels = train_test_split(train_images, train_labels, test_size=0.1, random_state=42)

# Train the model with reduced epochs
history = model.fit(small_train_images, small_train_labels, epochs=5, validation_split=0.2)


Epoch 1/5
125/125 [==============================] - 61s 55ms/step - loss: 2.2468 - accuracy: 0.1898 - val_loss: 2.2053 - val_accuracy: 0.2310
Epoch 2/5
125/125 [==============================] - 5s 44ms/step - loss: 2.1691 - accuracy: 0.2455 - val_loss: 2.1555 - val_accuracy: 0.2500
Epoch 3/5
125/125 [==============================] - 6s 49ms/step - loss: 2.1198 - accuracy: 0.2677 - val_loss: 2.1232 - val_accuracy: 0.2530
Epoch 4/5
125/125 [==============================] - 5s 42ms/step - loss: 2.0831 - accuracy: 0.2775 - val_loss: 2.0991 - val_accuracy: 0.2580
Epoch 5/5
125/125 [==============================] - 7s 55ms/step - loss: 2.0543 - accuracy: 0.2817 - val_loss: 2.0815 - val_accuracy: 0.2640


# Evaluate and Fine-Tune
After training, evaluate your model's performance on the test set:

In [22]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test Accuracy: {test_acc*100:.2f}%")

313/313 [==============================] - 11s 36ms/step - loss: 2.0841 - accuracy: 0.2624
Test Accuracy: 26.24%


# Fine-Tuning (Optional)


In [ ]:
base_model.trainable = True

# It's important to recompile the model after making any changes to the `trainable` attribute of any layer.
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),  # Very low learning rate
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history_fine = model.fit(train_images, train_labels, epochs=5, validation_data=(test_images, test_labels))

Epoch 1/5
 521/1563 [=========>....................] - ETA: 4:08 - loss: 3.5978 - accuracy: 0.1450

# Evaluate again

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test Accuracy: {test_acc*100:.2f}%")